In [1]:
from qiskit.circuit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit import transpile, assemble, Aer, IBMQ
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from sympy import Matrix
import matplotlib.pyplot as plt

import numpy as np

 Draper Adder 

First: QFT on the zero initialized result register, second: controlled Phase rotations as per the Draper scheme with one of the number register as control
finally: apply inverse QFT to get the original space.

In [2]:
def qft_rotations(circuit, n):
    
    if n == 0:
        return circuit
    n -= 1
    circuit.h(n)
    for qubit in range(n):
        circuit.cp(np.pi/2**(n-qubit), qubit, n)
    qft_rotations(circuit, n) 
    
def swap_registers(circuit, n):
   
    for qubit in range(n//2):
        circuit.swap(qubit, n-qubit-1)
    return circuit


def qft(circuit, n):
    
    qft_rotations(circuit, n)
    swap_registers(circuit, n)
    return circuit

In [3]:
def draper_scheme_iter(a_idx, b_idx, n, scale_factor=0):
    # Draper Scheme on the a with b, both numbers
    circuit = QuantumCircuit(a_idx + 1, name="Draper_adder")

    for a_qubit in range(b_idx + 1, a_idx + 1):
        for b_qubit in range(0, b_idx + 1):
            exp = a_qubit - b_idx - 1 + b_qubit + 1 - n
            if exp < 0:
                continue
            circuit.cp((np.pi/2**(a_qubit - b_idx - 1 + b_qubit + 1 - n)) * (2 ** scale_factor), b_qubit, a_qubit)
    return circuit

In [4]:
def inverse_qft(circuit, n):

    qft_circ = qft(QuantumCircuit(n), n)
    invqft_circ = qft_circ.inverse()
    circuit.append(invqft_circ, circuit.qubits[:n])
    return circuit.decompose()

### QFT Multiplier

In [5]:
def sub_adder(n, scale):
  
    circuit = QuantumCircuit(3*n, name="Drap_adder, 2^{}".format(scale))
    circuit.append(draper_scheme_iter(3*n-1, n-1, n,scale_factor=scale), circuit.qubits[:])
    return circuit.decompose()

#####  A Multiplication Example

In [6]:
a = 4
b = 5
n = (len(bin(b)) - 2) if len(bin(b)) >= len(bin(a)) else (len(bin(a)) - 2)
a_bin = np.binary_repr(a, width=n)
b_bin = np.binary_repr(b, width=n)
a_arr, b_arr = list(a_bin), list(b_bin)


In [7]:
q = QuantumRegister(4*n)
qc = QuantumCircuit(q)
for i in range(n):
    if b_arr[i] == '1':
        qc.x(i)
for i in range(n):
    if a_arr[i] == '1':
        qc.x(n + i)
qft_circ = qft(QuantumCircuit(2*n, name="QFT"), 2*n)

qc.barrier()
qc.append(qft_circ, q[4*n:2*n-1:-1])
qc.barrier()

In [8]:
for i in range(n):
    subadder_now = sub_adder(n, i).control(1)
    qc.append(subadder_now,  [q[n-1-i]] + q[n:])

In [9]:
qc.barrier()
inv_qft = inverse_qft(QuantumCircuit(2*n, name="INV_QFT"), 2*n)
qc.append(inv_qft.to_instruction(), q[4*n:2*n-1:-1])

qc.measure_all()
qc.draw(scale=0.5)

┌───┐ ░          ░                                             »
   q0_0: ┤ X ├─░──────────░─────────────────────────────────────────────»
         └───┘ ░          ░                       ┌────────────────────┐»
   q0_1: ──────░──────────░───────────────────────┤0                   ├»
         ┌───┐ ░          ░ ┌────────────────────┐│                    │»
   q0_2: ┤ X ├─░──────────░─┤0                   ├┤                    ├»
         ├───┤ ░          ░ │                    ││                    │»
   q0_3: ┤ X ├─░──────────░─┤1                   ├┤1                   ├»
         └───┘ ░          ░ │                    ││                    │»
   q0_4: ──────░──────────░─┤2                   ├┤2                   ├»
               ░          ░ │                    ││                    │»
   q0_5: ──────░──────────░─┤3                   ├┤3                   ├»
               ░ ┌──────┐ ░ │                    ││                    │»
   q0_6: ──────░─┤5     ├─░─┤4                   ├┤4 c_Drap_adder, 2^1 ├»
               ░ │      │ ░ │  c_Drap_adder, 2^0 ││                    │»
   q0_7: ──────░─┤4     ├─░─┤5                   ├┤5                   ├»
               ░ │      │ ░ │                    ││                    │»
   q0_8: ──────░─┤3     ├─░─┤6                   ├┤6                   ├»
               ░ │  QFT │ ░ │                    ││                    │»
   q0_9: ──────░─┤2     ├─░─┤7                   ├┤7                   ├»
               ░ │      │ ░ │                    ││                    │»
  q0_10: ──────░─┤1     ├─░─┤8                   ├┤8                   ├»
               ░ │      │ ░ │                    ││                    │»
  q0_11: ──────░─┤0     ├─░─┤9                   ├┤9                   ├»
               ░ └──────┘ ░ └────────────────────┘└────────────────────┘»
meas: 12/═══════════════════════════════════════════════════════════════»
                                                                        »
«         ┌────────────────────┐ ░              ░ ┌─┐                        »
«   q0_0: ┤0                   ├─░──────────────░─┤M├────────────────────────»
«         │                    │ ░              ░ └╥┘┌─┐                     »
«   q0_1: ┤                    ├─░──────────────░──╫─┤M├─────────────────────»
«         │                    │ ░              ░  ║ └╥┘┌─┐                  »
«   q0_2: ┤                    ├─░──────────────░──╫──╫─┤M├──────────────────»
«         │                    │ ░              ░  ║  ║ └╥┘┌─┐               »
«   q0_3: ┤1                   ├─░──────────────░──╫──╫──╫─┤M├───────────────»
«         │                    │ ░              ░  ║  ║  ║ └╥┘┌─┐            »
«   q0_4: ┤2                   ├─░──────────────░──╫──╫──╫──╫─┤M├────────────»
«         │                    │ ░              ░  ║  ║  ║  ║ └╥┘┌─┐         »
«   q0_5: ┤3                   ├─░──────────────░──╫──╫──╫──╫──╫─┤M├─────────»
«         │  c_Drap_adder, 2^2 │ ░ ┌──────────┐ ░  ║  ║  ║  ║  ║ └╥┘┌─┐      »
«   q0_6: ┤4                   ├─░─┤5         ├─░──╫──╫──╫──╫──╫──╫─┤M├──────»
«         │                    │ ░ │          │ ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   »
«   q0_7: ┤5                   ├─░─┤4         ├─░──╫──╫──╫──╫──╫──╫──╫─┤M├───»
«         │                    │ ░ │          │ ░  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐»
«   q0_8: ┤6                   ├─░─┤3         ├─░──╫──╫──╫──╫──╫──╫──╫──╫─┤M├»
«         │                    │ ░ │  INV_QFT │ ░  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘»
«   q0_9: ┤7                   ├─░─┤2         ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
«         │                    │ ░ │          │ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
«  q0_10: ┤8                   ├─░─┤1         ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
«         │                    │ ░ │          │ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
«  q0_11: ┤9                   ├─░─┤0         ├─░──╫──╫──╫──╫──╫──╫──╫──╫──╫─»
«         └────────────────────┘ ░ └──────────┘ ░  ║  ║  ║  ║  ║  ║  ║  ║  ║ »
«meas: 12/══════════════════════════

In [10]:
print("The depth of circuit: ", qc.depth())

The depth of circuit:  7
